#General informations


You can use a modifiable version of this code by going to folder/save a copy in the drive

---

Each **code blocks** or **code line** begins with its functionality and two parameters :

*   *User interaction* : **AUTO** = **DO NOT TOUCH THE CONTENT** / **MANU** = some parameters might be manually modified, by default, the code is functional.

*   *Importance of the block* : **NECESSARY** = the block must be run at least once for the analysis to success / **OPTIONAL** = no need to run the block

Sometime, within a code block, some options are deactivated by default and can be reactivated by the user if requiered.

---

**Interesting links**

Github : https://github.com/matheo-daly/highway_star 

Pypi : https://pypi.org/project/highway-star/

Version Mathéo 1 : https://colab.research.google.com/github/matheo-daly/highway_star/blob/main/highway_star_demo.ipynb

Version Mathéo 2 (english support) : https://colab.research.google.com/drive/1kXIZmzZ1ctKaL8ET7G9a5R31DYz4rp5E?usp=sharing

Version Mathéo 3 (Une fonction à run) : https://colab.research.google.com/drive/1mi9-uGGTuYy9CNwWS64vYX_Cve2uxMJa#scrollTo=B0S_v5LUOrPi

#Import

Must be run once at the begining of each session

In [1]:
# Import
# |AUTO|NECESSARY|

# highway-star = le code de Matéo
!pip install highway-star
from highway_star.preprocessing.biography_preprocessor import remove_stop_words_from_biographies
from highway_star.preprocessing.biography_preprocessor import sent_to_words
from highway_star.visualizing.visualizer import give_sankey_data_from_prefixspan
from highway_star.visualizing.visualizer import sankey_diagram_with_prefixspan_output

# Spacy = natural language processing ; pour lemmatisation, tout ça
!pip install https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-2.0.0/fr_core_news_sm-2.0.0.tar.gz#egg=fr_core_news_sm==2.0.0
!python -m spacy download fr

# nltk = natural language toolkit | utile pour text processing 2 
import nltk
nltk.download('stopwords')

# Panda: gestion database
import pandas as pd

# colab: gestion code-interface colab (pour import/export csv)
from google.colab import files

# import io pour transformer les .csv en string que panda peut lire
import io

print("import successful")

/Users/Aname/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Processing /Users/Aname/Library/Caches/pip/wheels/d2/a3/3a/25eb47393866089ed8e4e4e11a5e142e7b029c5aa8b52464b0/fr_core_news_sm-2.0.0-py3-none-any.whl
  Attempting uninstall: fr-core-news-sm
    Found existing installation: fr-core-news-sm 3.0.0
    Uninstalling fr-core-news-sm-3.0.0:
      Successfully uninstalled fr-core-news-sm-3.0.0
⚠ As of spaCy v3.0, shortcuts like 'fr' are deprecated. Please use the
full pipeline package name 'fr_core_news_sm' instead.
     |████████████████████████████████| 17.2 MB 3.1 MB/s eta 0:00:01


  Attempting uninstall: fr-core-news-sm
    Found existing installation: fr-core-news-sm 2.0.0
    Uninstalling fr-core-news-sm-2.0.0:
      Successfully uninstalled fr-core-news-sm-2.0.0
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


[nltk_data] Downloading package stopwords to /Users/Aname/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'google'

#1. Data input

Au choix, *1a. Wikipedia* **OU** *1b. csv upload*



##1a. Wikipedia scrapping

Récupère le contenu biographique d'une catégorie et le stock dans un dataframe prêt à être utilisé.


In [2]:
# Wikipedia Scrapping

## Wikipedia's category name, spaces replaced by underscores (e.g. "2020s suicides" become "2020s_suicides")
## |MANU|NECESSARY|
var_corpus_name = "Peintre_Flamand"

## Language of the Wikipedia scrapped. Option : "eng" or "fr"
## |MANU|NECESSARY|
var_language = "fr"

## La fonction qui récupère les données
## |AUTO|NECESSARY|
from highway_star.scrapping.wikipedia_scraper import scrap_wikipedia_structure_with_content
content = scrap_wikipedia_structure_with_content(
    root_category=var_corpus_name,
    lang=var_language)

## Stockage des données par panda
## |AUTO|NECESSARY|
dataframe_with_biographies = pd.DataFrame.from_dict(content)

## Affichage des données
## |MANU|OPTIONAL|
print(dataframe_with_biographies)

Output()

Output()

Empty DataFrame
Columns: [pages_links, pages_names, subcategory, content]
Index: []


### download csv


In [ ]:
# Dowload corpus as .csv
# |AUTO|OPTIONAL|

## Force mise à jour de panda pour gestion du process
## /!\ demande confirmation pour gestion panda
!pip uninstall pandas
!pip install pandas==1.1.5

## Nom du .csv
var_csv_name = var_corpus_name+".csv"

## Creation du .csv
dataframe_with_biographies.to_csv(var_csv_name) 
### Fichier peut être récupéré manuellement: onglet de gauche/symbole folder/clique droit/dowload
### /!\ Requière actualisation (en haut, flèche sur folder)

## Dl en code. Défaut = désactivé. Options: "0" (désactivé) ou "1" (activé)
var_dl_csv = 0
if var_dl_csv == 1 :
  files.download(var_csv_name)

## Remet Panda sur pieds
!pip install pandas --upgrade



Found existing installation: pandas 1.2.5
Uninstalling pandas-1.2.5:
  Would remove:
    /Users/Aname/opt/anaconda3/lib/python3.8/site-packages/pandas-1.2.5.dist-info/*
    /Users/Aname/opt/anaconda3/lib/python3.8/site-packages/pandas/*
Proceed (y/n)? 

##1b. Csv upload

upload un fichier .csv contenant les narrations et le stock dans un dataframe prêt à être utilisé.

**/!\** Un texte par "case" dans une colonne intitulée `content`. Le contenu des autres colonnes n'importe pas. Encodage = utf-8. Écrire le nom du fichier avant de lancer le code.

In [ ]:
# Upload .csv

## File name without extension
## |MANU|NECESSARY|
var_corpus_name = "typeD"

## Language of the corpus. Option : "eng" or "fr" (used for preprocessing)
## |MANU|NECESSARY|
var_language = "eng"

## Opens windows to select .csv from local computer
## |AUTO|NECESSARY|
uploaded = files.upload()

## Creation d'un dataframe à partir du contenu uploadé.
## |AUTO|NECESSARY|
var_file_name_w_ext = var_corpus_name + ".csv"
dataframe_with_biographies = pd.read_csv(io.StringIO(uploaded[var_file_name_w_ext].decode('utf-8')))

## Shows the dataframe
## |AUTO|OPTIONAL|
print(dataframe_with_biographies)

#2. Preprocessing

Formattage du corpus (tokenization, lemmatization, stopwords)

In [ ]:
# Corpus Preprocessing

## customs stopwords. ex. var_custom_stop_words = ["ajouter", "oui", "être", "avoir", "et", "d"]
## |MANU|OPTIONAL|
var_custom_stop_words = ["wer", "boire", "sentir", "avoir"]

## lemmatization. Option: (True) / (False)
## |MANU|NECESSARY|
var_use_lemmatization=True

## les classes grammaticales qu'on garde
## |MANU|NECESSARY|
var_allowed_postags=['NOUN', 'VERB']

## Les fonctions qui font le taf
## |AUTO|NECESSARY|
dataframe_with_biographies["biographies_tokenized"] = sent_to_words(biographies_column=dataframe_with_biographies["content"])
dataframe_with_biographies["biographies_preprocessed"] = remove_stop_words_from_biographies(
                                    biographies_column=dataframe_with_biographies["biographies_tokenized"], 
                                    custom_stop_words = var_custom_stop_words,
                                    use_lemmatization=var_use_lemmatization,
                                    allowed_postags=var_allowed_postags,
                                    lang=var_language)

## Montre les données qu'on va donner au sankey diagram
## |AUTO|OPTIONAL|
print(dataframe_with_biographies["biographies_preprocessed"])

Time to tokenize everything: 0.0 mins
Time to tokenize everything: 0.0 mins


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Output()

Output()

Series([], Name: biographies_preprocessed, dtype: float64)


### download csv


In [ ]:
# Dowload corpus as .csv
# |AUTO|OPTIONAL|

## Force mise à jour de panda pour gestion du process
## /!\ demande confirmation pour gestion panda
!pip uninstall pandas
!pip install pandas==1.1.5

## Nom du .csv
var_csv_name = var_corpus_name+".csv"

## Creation du .csv
dataframe_with_biographies.to_csv(var_csv_name) 
### Fichier peut être récupéré manuellement: onglet de gauche/symbole folder/clique droit/dowload
### /!\ Requière actualisation (en haut, flèche sur folder)

## Dl en code. Défaut = désactivé. Options: "0" (désactivé) ou "1" (activé)
var_dl_csv = 0
if var_dl_csv == 1 :
  files.download(var_csv_name)

## Remet Panda sur pieds
!pip install pandas --upgrade

#3. Visualization

Génère le diagramme de Sankey à partir du corpus.


In [ ]:
# Visualization

## Longueur de la chaine de mots
## |MANU|NECESSARY|
var_prefixspan_minlen = 5

## top combien on garde (les 10,100, etc. plus fréquents)
## |MANU|NECESSARY|
var_prefixspan_topk = 10

## La fonction qui fait le taf
## |AUTO|NECESSARY|
sankey_data_from_prefixspan = give_sankey_data_from_prefixspan(dataframe_with_biographies["biographies_preprocessed"],
                                prefixspan_minlen=var_prefixspan_minlen,
                                prefixspan_topk=var_prefixspan_topk)

## Affiche la table à l'origine du diagram
## |MANU|OPTIONAL|
print(sankey_data_from_prefixspan)

## Nomme le graphe pour que son contenu soit compréhensible
## |AUTO|NECESSARY|
var_sankey_name = var_corpus_name + "-str_len=" + str(var_prefixspan_minlen) + "-rlt_amt=" + str(var_prefixspan_topk)

## Create diagram
## |AUTO|NECESSARY|
sankey_diagram_with_prefixspan_output(sankey_data_from_prefixspan=sankey_data_from_prefixspan, 
                                        js_filename=var_sankey_name,
                                        html_filename=var_sankey_name,
                                        title=var_sankey_name)

## Les fichiers peuvent être récupéré dans les fichiers de la session du colab (onglet_gauche>Fichiers>Actualiser>le.csv et le .html)
## Ranger les 2 fichiers au meme endroit et ouvrir le html


Executing Prefixspan...
Time to execute prefixspan: 0.0 min


Output()

[]
js file written successfully !
html file written successfully !


In [ ]:
# Auto dl the diagram's files
# |AUTO|OPTIONAL|

files.download(var_sankey_name+".html")
files.download(var_sankey_name+".js")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Multiple diagram generator/download

In [ ]:
# Repetitive diagram generation and dowload
# |MANU|OPTIONAL|

## Range of the string lenght to be explored
## |MANU|NECESSARY|
var_minlen_min = 1
var_minlen_max = 4
var_minlen_step = 1

## Range of the numbers of kept relations
## |MANU|NECESSARY|
var_topk_min = 5
var_topk_max = 9
var_topk_step = 1

## Begining of the operation
## |AUTO|NECESSARY|
var_prefixspan_minlen = var_minlen_min
var_prefixspan_topk = var_topk_min

## Lancement de la boucle
## |AUTO|NECESSARY|
while var_prefixspan_minlen <= var_minlen_max :
  while var_prefixspan_topk <= var_topk_max :
    ### data conversion
    sankey_data_from_prefixspan = give_sankey_data_from_prefixspan(dataframe_with_biographies["biographies_preprocessed"],
                                    prefixspan_minlen=var_prefixspan_minlen,
                                    prefixspan_topk=var_prefixspan_topk)
    ### Nomme le graphe pour que son contenu soit compréhensible
    var_sankey_name = var_corpus_name + "-str_len=" + str(var_prefixspan_minlen) + "-rlt_amt=" + str(var_prefixspan_topk)
    ### Create diagram
    sankey_diagram_with_prefixspan_output(sankey_data_from_prefixspan=sankey_data_from_prefixspan, 
                                            js_filename=var_sankey_name,
                                            html_filename=var_sankey_name,
                                            title=var_sankey_name)
    ### Auto dl the diagram's files
    files.download(var_sankey_name+".html")
    files.download(var_sankey_name+".js")
    ### Le compteur de tour
    var_prefixspan_topk = var_prefixspan_topk + var_topk_step

  ## Set the indented boucle to the start
  var_prefixspan_topk = var_topk_min
  ## Le compteur de tour
  var_prefixspan_minlen = var_prefixspan_minlen + var_minlen_step


## download csv


In [ ]:
# Dowload Sankey relations as .csv file
## |AUTO|OPTIONAL|

## Converti la liste en dataframe (plus facilement convertible en .csv)
Sankey_relations_as_df = pd.DataFrame(sankey_data_from_prefixspan)
## affiche le dataframe
print(Sankey_relations_as_df)

## Force mise à jour de panda pour gestion du process
## /!\ demande confirmation pour gestion panda
!pip uninstall pandas
!pip install pandas==1.1.5

## Nom du .csv
var_csv_name = var_sankey_name + ".csv"

## Creation du .csv
Sankey_relations_as_df.to_csv(var_csv_name) 
### Fichier peut être récupéré manuellement: onglet de gauche/symbole folder/clique droit/dowload
### /!\ Requière actualisation (en haut, flèche sur folder)

## Dl en code. Défaut = désactivé. Options: "0" (désactivé) ou "1" (activé)
var_dl_csv = 0
if var_dl_csv == 1 :
  files.download(var_csv_name)

## Remet Panda sur pieds
!pip install pandas --upgrade

         0       1   2
0     born  family  45
1   family  became  51
2    began   first  15
3    first     one   5
4      one    made   5
5     made     thi  10
6    first     wer  10
7      wer    made   5
8      wer     one   5
9      one     thi   5
10    born  became   5
11  family   later   5
12     wer    said   4
Found existing installation: pandas 1.3.2
Uninstalling pandas-1.3.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/pandas-1.3.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pandas/*
Proceed (y/n)? y
  Successfully uninstalled pandas-1.3.2
  Using cached pandas-1.1.5-cp37-cp37m-manylinux1_x86_64.whl (9.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
highway-star 0.0.8.1 requires pandas~=1.2.4, but you have pandas 1.1.5 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.25.1 wh

  Using cached pandas-1.3.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
highway-star 0.0.8.1 requires pandas~=1.2.4, but you have pandas 1.3.2 which is incompatible.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.2 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.25.1 which is incompatible.
